## Imports

In [45]:
from openseize.file_io import bases
from openseize.file_io import edf
from openseize.demos import paths
import matplotlib.pyplot as plt

## Introduction

<font size=3>Currently, there is no agreed upon format for EEG files. Openseize can build producers from an EDF reader, numpy arrays or numpy memmapped arrays. <font color='darkcyan'><b>This tutorial provides guidance to users and developers who wish to use Openseize with non-EDF file types.</b></font> There are two main considerations; the type of file you want to read and the size of your files. This guide is broken down into the following sections:

- [Data Acquisition Systems using EDFs](#Data-Acquisition-Systems-using-EDFs)
- [Array to NDArray Conversion](#Array-to-NDArray-Conversion)
- [File-to-File Conversion](#File-to-File-Conversion)
- [Developing New Readers](#Developing-New-Readers)

## Data Acquisition Systems using EDFs

<font size=3>EDF files were the first file type chosen for Openseize support because they are used by many [data acquisition systems](https://www.edfplus.info/companies/index.html) and can be iteratively read since they are simple binary files. If you are an experimentalist working with Openseize, we first advise you to check the list above to determine if your data acquisition system supports exporting to the EDF file type.

## Array to NDArray Conversion

<font size=3>Openseize's producers can produce from ndarrays and numpy memmaps. If you can convert to an array type, Openseize can work with it. For example, users who have stored data to <font color='firebrick'>*.mat files </font> have two options depending on the file size. For small files you can use <font color='firebrick'>scipy.io.loadmat</font> to convert your *.mat file to an in-memory ndarray. For large files, [saved with the 7.3 mat version flag](https://www.mathworks.com/help/matlab/large-mat-files.html) you can use the [h5py library](https://www.h5py.org/) to read the files to a numpy memmap or [build an HDF5 reader](#Developing-New-Readers).

## File to File Conversion

### Using Openseize's edf.Writer

<font size=3>In addition to an EDF Reader, Openseize has an EDF Writer that can write EDF files given a header mapping and a data source. This source may be an ndarray, a numpy memmap, or a reader instance with a read method. The writers only publicly available method is the 'write' method:

In [11]:
help(edf.Writer.write)

Help on function write in module openseize.file_io.edf:

write(self, header: openseize.file_io.edf.Header, data: Union[numpy.ndarray, openseize.file_io.edf.Reader], channels: Sequence[int], verbose: bool = True) -> None
    Write header metadata and data for channel in channels to this
    Writer's file instance.
    
    Args:
        header:
            A mapping of EDF compliant fields and values.
        data:
            An array with shape (channels, samples) or Reader instance.
        channels:
            Channel indices to write to this Writer's open file.
        verbose:
            An option to print progress of write.
    
    Raises:
        ValueErrror: An error occurs if samples to be written is not
                     divisible by the number of records in the Header
                     instance.



<font size=3>This method requires both data and a header instance. The header is an extended dict that contains all the fields expected of an EDF compliant header. Here is the example header from our demo data:

In [27]:
# This is an example header from the demo file
fp = paths.locate('recording_001.edf')
reader = edf.Reader(fp)
print(reader.header)

{'version': '0',
 'patient': 'PIN-42 M 11-MAR-1952 Animal',
 'recording': 'Startdate 15-AUG-2020 X X X',
 'start_date': '15.08.20',
 'start_time': '09.59.15',
 'header_bytes': 1536,
 'reserved_0': 'EDF+C',
 'num_records': 3775,
 'record_duration': 1.0,
 'num_signals': 5,
 'names': ['EEG EEG_1_SA-B', 'EEG EEG_2_SA-B', 'EEG EEG_3_SA-B',
           'EEG EEG_4_SA-B', 'EDF Annotations'],
 'transducers': ['8401 HS:15279', '8401 HS:15279', '8401 HS:15279',
                 '8401 HS:15279', ''],
 'physical_dim': ['uV', 'uV', 'uV', 'uV', ''],
 'physical_min': [-8144.31, -8144.31, -8144.31, -8144.31, -1.0],
 'physical_max': [8144.319, 8144.319, 8144.319, 8144.319, 1.0],
 'digital_min': [-8192.0, -8192.0, -8192.0, -8192.0, -32768.0],
 'digital_max': [8192.0, 8192.0, 8192.0, 8192.0, 32767.0],
 'prefiltering': ['none', 'none', 'none', 'none', ''],
 'samples_per_record': [5000, 5000, 5000, 5000, 1024],
 'reserved_1': ['', '', '', '', '']}

{'Accessible Properties': ['annotated', 'annotation', 'chann

<font size=3> You can pass a header dict to the Header.from_dict method to create an edf compliant header for the Writer's write method.

In [26]:
help(edf.Header.from_dict)

Help on method from_dict in module openseize.file_io.edf:

from_dict(dic: Dict) -> 'Header' method of builtins.type instance
    Alternative constructor for creating a Header from a bytemap.
    
    Args:
        dic:
            A dictionary containing all expected bytemap keys.



<font size=3>Once you've built your header you can supply this along with a data array to the write method to write a new EDF file.

### Using MNE-Python io.export_raw

<font size=3>Several packages provide file conversion. The MNE-Python package can read a large variety of file types with its [read_raw](https://mne.tools/dev/generated/mne.io.read_raw.html#mne.io.read_raw) function. Once read, MNE has an [export_raw](https://mne.tools/dev/generated/mne.export.export_raw.html#mne.export.export_raw) function that can export to the EDF file type. **Please be aware that the exporter in MNE loads the data before writing** so this option is not suitable for large data files.

## Developing New Readers

<font size=3>One of the important aims in Openseize's development is extensibility. In the context of Openseize's Readers, this is achieved by defining the exact methods you must supply in order to build a reader that can be produced from. The *Abstract Base Class* Reader is the mechanism which checks that you have implemented all the required methods. Lets take a look at these required methods.

In [47]:
help(bases.Reader.channels)

Help on property:

    Returns the channels that this Reader will read.

